In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity

# 문제에 대한 정보
problem = pd.read_csv('tag_key.csv')

In [45]:
# 백준에서 사용자들의 푼 문제들에 대한 정보
user = pd.read_csv('userSolved.csv')

In [46]:
user

,userId,solved
0,koosaga,1000
1,koosaga,1001
2,koosaga,1002
3,koosaga,1003
4,koosaga,1004
...,...,...
8224055,ou0911,10998
8224056,ou0911,11654
8224057,ou0911,16394
8224058,ou0911,16430


In [47]:
problem.shape

(5598, 195)

In [48]:
problem

,Unnamed: 0,problemId,level,math,implementation,dp,graphs,data_structures,string,greedy,...,top_tree,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis
0,0,1000,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1001,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1271,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1550,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,2338,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5593,5593,21837,10,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
5594,5594,21855,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5595,5595,21917,10,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5596,5596,21937,10,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# 사용할 Column
# problemid, level, averageTries, tag-key
# 데이터 전처리
problem = problem.iloc[:,1:]
problem.head()

,problemId,level,math,implementation,dp,graphs,data_structures,string,greedy,bruteforcing,...,top_tree,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis
0,1000,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1271,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1550,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2338,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
# 새로운 피벗 테이블을 사용하기 위해 Column 명을 변경
user.columns = ['userId', 'problemId']

In [51]:
# 데이터 중복행 삭제
problem = problem.drop_duplicates()
# K-Means를 하기 위해 문제 아이디를 제거
problem_no = problem.iloc[:,1:]
problem.head()

,problemId,level,math,implementation,dp,graphs,data_structures,string,greedy,bruteforcing,...,top_tree,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis
0,1000,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1271,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1550,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2338,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# 30개의 단계로 구성되었으므로 30개로 작성
k = 30
model = KMeans(n_clusters=k, random_state=10)
model.fit(problem_no)

KMeans(n_clusters=30, random_state=10)

In [53]:
# Cluseter의 결과를 저장
problem['cluster'] = model.fit_predict(problem_no)

In [54]:
problem.tail()

,problemId,level,math,implementation,dp,graphs,data_structures,string,greedy,bruteforcing,...,chordal_graph,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis,cluster
5593,21837,10,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,20
5594,21855,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
5595,21917,10,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,5
5596,21937,10,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
5597,21959,10,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,26


In [55]:
# 사용자와 문제정보를 병합
user_problem = pd.merge(user, problem, on="problemId")

In [56]:
# 사용자와 문제에 대한 피벗 테이블을 생성
user_problem_pi = user_problem.pivot_table('cluster', index='problemId', columns='userId')

In [57]:
# 사용자들이 한번도 풀지 않은 문제에 대해 -1로 결측값을 처리
user_problem_pi.fillna(-1,inplace=True)
user_problem_pi.head()

userId,0000000000,0000064,0000sb,0004poppop,000ian,001,001dltmdals,0021,006ws,007001as,...,zzzwldy,zzzxc123,zzzz01111,zzzz465,zzzzddz,zzzzll,zzzzong123,zzzzz9887,zzzzzzzz,zzzzzzzzu2
problemId,,,,,,,,,,,,,,,,,,,,,
1000,24.0,24.0,24.0,24.0,-1.0,24.0,-1.0,24.0,24.0,24.0,...,24.0,24.0,24.0,24.0,-1.0,24.0,24.0,24.0,24.0,24.0
1001,24.0,24.0,24.0,24.0,-1.0,24.0,-1.0,-1.0,24.0,-1.0,...,24.0,24.0,24.0,24.0,-1.0,24.0,-1.0,24.0,24.0,24.0
1002,12.0,-1.0,-1.0,-1.0,-1.0,12.0,-1.0,12.0,-1.0,-1.0,...,12.0,-1.0,-1.0,12.0,-1.0,12.0,-1.0,-1.0,12.0,-1.0
1003,27.0,-1.0,-1.0,-1.0,-1.0,27.0,-1.0,27.0,27.0,-1.0,...,27.0,-1.0,-1.0,27.0,-1.0,27.0,-1.0,-1.0,-1.0,-1.0
1004,-1.0,-1.0,-1.0,-1.0,-1.0,27.0,-1.0,27.0,27.0,-1.0,...,27.0,-1.0,-1.0,-1.0,-1.0,27.0,-1.0,-1.0,-1.0,-1.0


In [58]:
# 문제간의 유사도를 구해 새로운 matiricx생성
problem_sim = cosine_similarity(user_problem_pi,user_problem_pi).argsort()[:,::-1]
print(problem_sim.shape)

(4870, 4870)


In [64]:
def Cal_distance(result, target_index):
        sorter = []
        for i in range(len(result)):
            sorter.append(pow(result.iat[i,1] - problem.iat[target_index[0],1],2))
        result['sorter'] = sorter
        return result

In [65]:
def get_recommand_problem(problemId, top=30):
    target_problem_index = problem[problem['problemId'] == problemId].index.values

    # 코사인 유사도중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = problem_sim[target_problem_index, :top].reshape(-1)
    # 본인 제외
    sim_index = sim_index[sim_index != target_problem_index]

    result = problem.iloc[sim_index]
    result = Cal_distance(result, target_problem_index)
    return result.sort_values('sorter')[:5]


In [66]:
get_recommand_problem(1000)

/var/folders/pd/b0c8c2p11k3d3g_6p6mfk28w0000gn/T/ipykernel_46372/1765496142.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['sorter'] = sorter


,problemId,level,math,implementation,dp,graphs,data_structures,string,greedy,bruteforcing,...,majority_vote,a_star,rb_tree,birthday,degree_sequence,discrete_kth_root,multipoint_evaluation,differential_cryptanalysis,cluster,sorter
1,1001,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,24,0
34,11283,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
5,2475,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,24,0
97,8723,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,24,1
430,11772,3,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,4


In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from sklearn.metrics.pairwise import cosine_similarity

def init():
    # 문제에 대한 정보
    problem = pd.read_csv('data/tag_key.csv')
    user = pd.read_csv('data/userSolved.csv')
    user.columns = ['userId', 'problemId']

    k = 50
    model = KMeans(n_clusters=k, random_state=10)
    model.fit(problem_no)

    # Cluseter의 결과를 저장
    problem['cluster'] = model.fit_predict(problem_no)

    # 사용자와 문제정보를 병합
    user_problem = pd.merge(user, problem, on="problemId")

    # 사용자와 문제에 대한 피벗 테이블을 생성
    user_problem_pi = user_problem.pivot_table('cluster', index='problemId', columns='userId')

    # 백준에는 있지만 Solved ac에는 없는 문제를 -1로 태깅
    user_problem_pi.fillna(-1,inplace=True)

    # 문제간의 유사도를 구해 새로운 matiricx생성
    problem_sim = cosine_similarity(user_problem_pi,user_problem_pi).argsort()[:,::-1]



def Cal_distance(result, target_index):
        sorter = []
        for i in range(len(result)):
            sorter.append(pow(result.iat[i,1] - self.problem.iat[target_index,1],2))
        result['sorter'] = sorter
        return result
     
def get_recommand_problem(problemId, top=30):
    for i in range(len(problem)):
        if problem['problemId'][i] == problemId:
            target_problem_index = i
            break

    # 코사인 유사도중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
    sim_index = self.problem_sim[target_problem_index, :top].reshape(-1)
    # 본인 제외
    sim_index = sim_index[sim_index != target_problem_index]

    result = problem.iloc[sim_index]
    result = Cal_distance(result, target_problem_index)
    return result.sort_values('sorter')[:5]


In [113]:
problem_sim

array([[   0,    1,  421, ...,  890, 1459, 3755],
       [   1,    0, 2179, ...,  890,  889, 3755],
       [   2,  596,  433, ..., 1387, 1459, 3755],
       ...,
       [4867, 4865, 4765, ...,  484,    0,  421],
       [4868, 4869, 4801, ...,  484,    0,  421],
       [4869,  919, 1589, ...,  484,    0,  421]])

In [114]:
type(problem['problemId'][1])

numpy.int64

In [115]:
#하나의 클래스로 진행
class Recommand:
    
    def __init__(self):
        # 사용자와 문제 정보를 읽어 저장
        self.problem = pd.read_csv('data/tag_key.csv')
        self.user = pd.read_csv('data/userSolved.csv')
        # 의미 없는 열 제거
#         self.problem = self.problem.iloc[:,1:]
        # 새로운 피벗 테이블을 사용하기 위해 Column 명을 변경
        self.user.columns = ['userId', 'problemId']
        # 데이터 중복행 삭제
        self.problem = self.problem.drop_duplicates()
    
    def Start_KMeans(self):
        # K-Means를 하기 위해 문제 아이디를 제거
        problem_no = self.problem.iloc[:,1:]
        # Cluster는 Solved.ac의 난이도의 수로 설정
        k = 30
        # 모델을 생성
        model = KMeans(n_clusters=k, random_state=10)
        # K-Means를 실행
        model.fit(problem_no)
        self.problem['cluster'] = model.fit_predict(problem_no)
        # 사용자와 문제정보를 병합
        user_problem = pd.merge(self.user, self.problem, on="problemId")
        # 사용자와 문제에 대한 피벗 테이블을 생성
        user_problem_pi = user_problem.pivot_table('cluster', index='problemId', columns='userId')
        # 백준에는 있지만 Solved ac에는 없는 문제를 -1로 태깅
        user_problem_pi.fillna(-1,inplace=True)
        # 사용자와 문제정보를 병합
        self.problem_sim = cosine_similarity(user_problem_pi, user_problem_pi).argsort()[:,::-1]
        
    def Cal_distance(self, result, target_index):
        sorter = []
        for i in range(len(result)):
            sorter.append(pow(result.iat[i,1] - self.problem.iat[target_index,1],2))
        result['sorter'] = sorter
        return result
     
    def get_recommand_problem(self, problemId, top=30):
        for i in range(len(self.problem)):
            if self.problem['problemId'][i] == problemId:
                target_problem_index = i
                break
        
        # 코사인 유사도중 비슷한 코사인 유사도를 가진 정보를 뽑아낸다.
        sim_index = self.problem_sim[target_problem_index, :top].reshape(-1)
        # 본인 제외
        sim_index = sim_index[sim_index != target_problem_index]
    
        result = self.problem.iloc[sim_index]
        result = self.Cal_distance(result, target_problem_index)
        return result.sort_values('sorter')[:5]


In [116]:
rc = Recommand()
rc.Start_KMeans()

ValueError: could not convert string to float: '  implementation    arithmetic    math  '

In [ ]:
rc.get_recommand_problem(1001)